In [ ]:
import sys
sys.path.append('..')
import os

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import re
import time
from tqdm import tqdm
import re
from llm.llm.gemini import Gemini
from llm.llm_utils import get_code_from_text_response, get_json_from_text_response
llm = Gemini()
import json


/Users/duongphuonggiang/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class JobScraperr:
    def __init__(self, base_url, output_csv):
        self.base_url = base_url
        self.output_csv = output_csv
        self.driver = self._initialize_driver()

        self.df = pd.DataFrame(columns=[
            "Job ID", "Title", "Link", "Job_Type", "Job_Location", 
            "Job_Category", "Salary", "Experience_Level", "Years_of_Experience",
            "Main_Job_Description", "Candidate_Experience_Requirements",  "Candidate_soft_skill_Requirements", "Candidate_technical_Requirements", "Candidate_degree_Requirements","Benefits", 
            "Additional_Notes", "Industry", "Address", "Academic_Degree", 
            "Sex_Requirement", "Degree_Level", 
            "Age_Requirement"
        ])
        if not os.path.exists(output_csv):
            self.df.to_csv(output_csv, index=False)

    def _initialize_driver(self):
        chrome_options = Options()
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--headless")

        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        return driver

    def clean_text(self, text):
        """Clean and standardize extracted text."""
        text = re.sub(r'<[^>]+>', '', text)  
        text = text.replace('\n', ' ').replace('<br>', ' ').replace('/', ' ').replace("*",'').replace('•','')
        text = re.sub(r'\s+', ' ', text).strip()  
        return text

    def get_job_details_from_gemini(self, html_content):
        prompt = (
            f"The following HTML contains a job posting:\n\n{html_content}\n\n"
            f"Extract the full text content of each specified section in JSON format. Clean the data by removing any unnecessary characters such as '/', '\\n', '<br>', and other HTML tags, bullet points (e.g., '-', '*', '•'), and other escape sequences. "
            f"Ensure that the content in each field uses ':' to separate key-value relationships (e.g., 'Key: Value') and ';' to separate items in a list (e.g., 'Item1; Item2; Item3'). "
            f"Return the content as plain text without formatting or unnecessary characters. If a section is missing, return 'Not available' as the value.\n\n"
            f"Combine the 'Years of Experience' and 'Experience Level' fields into 'Candidate Experience Requirements' if they exist. "
            f"If 'Candidate Experience Requirements' already contains content, append 'Years of Experience' and 'Experience Level' to it. If no data exists for these fields, leave the original field as is.\n\n"
            f"Expected JSON output:\n"
            f"{{\n"
            f"    \"Job_Type\": \"\",\n"
            f"    \"Job_Location\": \"\",\n"
            f"    \"Job_Category\": \"\",\n"
            f"    \"Main_Job_Description\": \"\",\n"
            f"    \"Experience_Level\": \"\",\n"
            f"    \"Years_of_Experience\": \"\",\n"
            f"    \"Candidate_Experience_Requirements\": \"Candidate_Experience_Requirement; Years_of_Experience: [Value] and Experience_Level: [Value]\" if available, else keep original content,\n"
            f"    \"Candidate_soft_skill_Requirements\": \"\",\n"
            f"    \"Candidate_technical_skill_Requirements\": \"\",\n"
            f"    \"Candidate_degree_Requirements\": \"\",\n"
            f"    \"Benefits\": \"\",\n"
            f"    \"Additional_Notes\": \"\",\n"
            f"    \"Industry\": \"\",\n"
            f"    \"Salary\": \"\",\n"
            f"    \"Address\": \"\",\n"
            f"    \"Academic_Degree\": \"\",\n"
            f"    \"Sex_Requirement\": \"\",\n"
            f"    \"Degree_Level\": \"\",\n"
            f"    \"Age_Requirement\": \"\",\n"
            f"    \"Job_Date-open\": \"\",\n"
            f"    \"Job_Date-end\": \"\"\n"
            f"}}"
        )


        message = [
            {
                'role': 'system',
                'content': 'You must extract the content in JSON format and translate into english'
            },
            {
                'role': 'user',
                'content': prompt
            }
        ]

        response = llm(message) 
        temp_cv = self.get_json_from_text_response(response)
        return temp_cv

    def get_json_from_text_response(self, response):
        """Extract JSON from text response."""
        cleaned_response = response.strip().lstrip("```json").rstrip("```").strip()

        try:
            json_data = json.loads(cleaned_response)
            if isinstance(json_data, dict):
                return {k: self.clean_text(v) if isinstance(v, str) else v for k, v in json_data.items()}
            else:
                return self._default_job_detail()
        except json.JSONDecodeError:
            print("Failed to parse JSON from response.")
            return self._default_job_detail()

    def _default_job_detail(self):
        return {col: 'Not available' for col in self.df.columns if col not in ["Job ID", "Title", "Link"]}

    def extract_job_detail(self, job_link, job_data):
        self.driver.execute_script("window.open(arguments[0], '_blank');", job_link)
        self.driver.switch_to.window(self.driver.window_handles[1])
        time.sleep(2)

        try:
            container_wrap = self.driver.find_element(By.CSS_SELECTOR, "div.container-wrap")
            header = self.driver.find_element(By.CSS_SELECTOR, "header.noo-page-heading")
            combined_html = header.get_attribute("outerHTML") + container_wrap.get_attribute("outerHTML")

            job_detail = self.get_job_details_from_gemini(combined_html) 
            
            job_detail.update(job_data)

            if job_detail:
                self.append_to_df(job_detail)

            self.driver.close()
            self.driver.switch_to.window(self.driver.window_handles[0])
        except Exception as e:
            print(f"Failed to extract job detail for link {job_link}: {e}")
            self.driver.close()
            self.driver.switch_to.window(self.driver.window_handles[0])

    def append_to_df(self, job_detail):
        job_df = pd.DataFrame([job_detail])
        self.df = pd.concat([self.df, job_df], ignore_index=True)
        print("Appended job detail to DataFrame.")

    def extract_jobs_from_page(self, job_elements):
        for job_element in job_elements:
            try:
                job_id = job_element.find_element(By.CSS_SELECTOR, "a.btn-quick-view-popup").get_attribute("data-id")
                title = job_element.find_element(By.CSS_SELECTOR, "h3 a").text.strip()
                job_link = job_element.find_element(By.CSS_SELECTOR, "a.job-details-link").get_attribute("href")

                job_data = {
                    "Job ID": job_id,
                    "Title": title,
                    "Link": job_link,
                }

                self.extract_job_detail(job_link, job_data)
                print(f"Processed job ID: {job_id}")
            except Exception as e:
                print(f"Error extracting job: {e}")

    def crawl_jobs(self, start_page, end_page):
        for page in range(start_page, end_page + 1):
            url = f"{self.base_url}{page}/"
            self.driver.get(url)
            time.sleep(1)

            try:
                WebDriverWait(self.driver, 5).until(
                    EC.visibility_of_all_elements_located((By.CSS_SELECTOR, ".noo-job-item.noo_job"))
                )
                job_elements = self.driver.find_elements(By.CSS_SELECTOR, ".noo-job-item.noo_job")
                print(f"Found {len(job_elements)} jobs on page {page}")
                
                self.extract_jobs_from_page(job_elements)
            except Exception as e:
                print(f"Error loading page {page}: {e}")

            time.sleep(1)

        self.driver.quit()
        self.save_to_csv()
        print(f"Data saved to {self.output_csv}")

    def save_to_csv(self):
        self.df.to_csv(self.output_csv, index=False)
        print("Data saved to CSV.")


In [4]:
scraper = JobScraperr(base_url="https://topjobvn.com/vi/jobs/page/", output_csv="test_topjob.csv")
scraper.crawl_jobs(start_page=1, end_page=20)

2024-11-26 17:38:50,108 - INFO - ====== WebDriver manager ======
2024-11-26 17:38:50,410 - INFO - Get LATEST chromedriver version for google-chrome
2024-11-26 17:38:50,656 - INFO - Get LATEST chromedriver version for google-chrome
2024-11-26 17:38:50,830 - INFO - Driver [/Users/duongphuonggiang/.wdm/drivers/chromedriver/mac64/131.0.6778.85/chromedriver-mac-arm64/chromedriver] found in cache


Found 10 jobs on page 1


2024-11-26 17:39:10,161 - INFO - Completion time of gemini-1.5-flash-002: 8.55342698097229s


prompt_token_count: 81003
candidates_token_count: 381
total_token_count: 81384

Appended job detail to DataFrame.
Processed job ID: 61009


2024-11-26 17:39:22,180 - INFO - Completion time of gemini-1.5-flash-002: 9.840788841247559s


...

prompt_token_count: 80881
candidates_token_count: 518
total_token_count: 81399

Appended job detail to DataFrame.
Processed job ID: 43915


2024-11-26 18:19:16,662 - INFO - Completion time of gemini-1.5-flash-002: 7.095476865768433s


prompt_token_count: 78930
candidates_token_count: 391
total_token_count: 79321

Appended job detail to DataFrame.
Processed job ID: 43194


2024-11-26 18:19:27,841 - INFO - Completion time of gemini-1.5-flash-002: 8.476579904556274s


prompt_token_count: 78931
candidates_token_count: 568
total_token_count: 79499

Appended job detail to DataFrame.
Processed job ID: 57671
Data saved to CSV.
Data saved to test_topjob.csv


In [35]:
df = scraper.df


In [38]:

placeholders = ['Not available', 'No requirement', 'Not required', "No experience", "No degree", "No certificate","not available"]
df.replace(placeholders, "No requirement", inplace=True)

def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text)
    return text.replace("\n", " ").replace("\t", " ").strip().lower()

columns_to_clean = [
    "Candidate_Experience_Requirements",
    "Candidate_soft_skill_Requirements",
    "Candidate_technical_skill_Requirements",
    "Candidate_degree_Requirements"
]

for col in columns_to_clean:
    if col in df.columns:
        df[col] = df[col].apply(clean_text)

date_columns = ["Job_Date-open", "Job_Date-end"]
for col in date_columns:
    if col in df.columns:
        df[col] = df[col].str.replace(" ", "-", regex=False)

def extract_min_salary(salary):
    if pd.isna(salary):
        return pd.NA
    numbers = [int(num.replace(',', '')) for num in salary.split() if num.replace(',', '').isdigit()]
    return numbers[0] if numbers else pd.NA

df['Salary_Min'] = df['Salary'].apply(extract_min_salary)
df['Salary_Min'] = pd.to_numeric(df['Salary_Min'], errors='coerce').astype('Int64')


In [33]:
df.to_csv("topjob.csv")

In [10]:
import pandas as pd
import json
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
tokenizer = AutoTokenizer.from_pretrained("BAAI/llm-embedder")
model_embed = AutoModel.from_pretrained("BAAI/llm-embedder")
model_embed.eval()

def last_token_pool(last_hidden_states: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])  
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

def embed_text(text):
    if pd.isna(text) or text.lower().strip() == "no requirement":
        return "no requirement"  
    inputs = tokenizer(str(text), return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        out = model_embed(**inputs)
        embeddings = last_token_pool(out.last_hidden_state, inputs['attention_mask'])
    embeddings = F.normalize(embeddings, p=2, dim=1)
    return embeddings.tolist()

file_path = '/Users/duongphuonggiang/Documents/VietCvProcessor/crawl/topjob.csv'
df = pd.read_csv(file_path)

In [15]:
import os
from tqdm import tqdm
import pandas as pd
import json

placeholders = ['Not available', 'No requirement', 'Not required', "No experience", "No degree", "No certificate"]
df.replace(placeholders, pd.NA, inplace=True)

def generate_embedding_for_column(df, col_name):
    tqdm.pandas(desc=f"Embedding {col_name}")
    return df[col_name].progress_apply(embed_text)

df["Candidate_Experience_Requirements_embedding"] = generate_embedding_for_column(df, "Candidate_Experience_Requirements")
df["Candidate_soft_skill_Requirements_embedding"] = generate_embedding_for_column(df, "Candidate_soft_skill_Requirements")
df["Candidate_technical_skill_Requirements_embedding"] = generate_embedding_for_column(df, "Candidate_technical_skill_Requirements")
df["Candidate_degree_Requirements_embedding"] = generate_embedding_for_column(df, "Candidate_degree_Requirements")

def create_mappings(df):
    mappings = []
    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Creating mappings"):
        job_info = {
            "JOB_ID": row.get("Job ID"),
            "location": row.get("Job_Location"),
            "date_opened": row.get("Job_Date-open"),
            "deadline": row.get("Job_Date-end"),
            "title": row.get("Title"),
            "description": row.get("Candidate_Experience_Requirements"),
            "companyID": row.get("Job ID"),
            "job information": {
                "experience_embedding": row.get("Candidate_Experience_Requirements_embedding", "no requirement"),
                "soft_skill_embedding": row.get("Candidate_soft_skill_Requirements_embedding", "no requirement"),
                "technical_skill_embedding": row.get("Candidate_technical_skill_Requirements_embedding", "no requirement"),
                "degree_embedding": row.get("Candidate_degree_Requirements_embedding", "no requirement")
            },
            "workingType": row.get("Job_Type"),
            "workingTime": row.get("Job_Category"),
            "salary_month": row.get("Salary_Min"),
            "sectorID": row.get("Job ID"),
        }
        mappings.append({"mappings": {"properties": job_info}})
    
    return mappings

job_mappings = create_mappings(df)

output_dir = 'crawl'
output_file = os.path.join(output_dir, 'job_mappings.json')
os.makedirs(output_dir, exist_ok=True)


Embedding Candidate_Experience_Requirements:   0%|          | 0/200 [00:00<?, ?it/s]

Embedding Candidate_Experience_Requirements:  16%|█▌        | 31/200 [00:02<00:13, 12.57it/s]


KeyboardInterrupt: 